In [ ]:
%matplotlib inline

from matplotlib import pyplot as plt

plt.rcParams['figure.figsize'] = [10, 10]
plt.rcParams['image.origin'] = 'lower'
#plt.rcParams.update({'font.size': 17})

SMALL_SIZE = 15
MEDIUM_SIZE = 18
BIGGER_SIZE = 20

#plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

def savefig(filename):
    plt.savefig("plots/"+filename, dpi=70, bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
from astropy.nddata import CCDData, Cutout2D
from astropy.io import fits 
from astropy import units as u 

In [ ]:
url = 'https://archive.stsci.edu/pub/hlsp/frontier/abell2744/images/hst/v1.0/hlsp_frontier_hst_wfc3-60mas_abell2744_f105w_v1.0_drz.fits'
image_data = CCDData.read(url, cache=True)
rms_data = fits.getdata(url.replace('drz', 'rms'))

In [ ]:
from astropy.coordinates import SkyCoord

targ_x, targ_y = image_data.wcs.world_to_pixel(SkyCoord(3.596248*u.deg,  -30.388517*u.deg))

In [ ]:
from astropy import units as u 

image = Cutout2D(image_data, position=(targ_x-1, targ_y), size=150, copy=True, wcs=image_data.wcs)
rms =  Cutout2D(rms_data, position=(targ_x-1, targ_y), size=150, copy=True)

In [ ]:
import numpy as np

vmax = 0.05 # Use the image std as max and min of all plots 
vmin = - vmax 

plt.imshow(image.data, vmin=vmin, vmax=vmax)
plt.title("Galaxy in Abell 2744")
plt.xlabel("Pixels")
plt.ylabel("Pixels")
plt.show()

In [ ]:
from petrofit.fitting import fit_background
import numpy as np 

def calc_bg(target_image):
    fig, axs = plt.subplots(1,3)
    x_arr = np.linspace(0, target_cutout_size, target_cutout_size+1)[:-1]
    x_grid, y_grid = np.meshgrid(x_arr, x_arr)
    
    bg_image = fit_background(target_image, sigma=3.0)[0](x_grid, y_grid)
    
    plt.sca(axs[0])
    plt.imshow(bg_image)
    
    plt.sca(axs[1])
    plt.imshow(bg_image, vmax=vmax, vmin=vmin)
    
    plt.sca(axs[2])
    plt.imshow(target_image-bg_image, vmax=vmax, vmin=vmin)
    
    return bg_image

target_cutout_size = len(image.data)
bg = calc_bg(image.data)
image.data = image.data - bg 

## Save Image and RMS 

In [ ]:
image_header = image_data.header.copy()
image_header.update(image.wcs.to_header())
image_header.update({
    'EXPTIME' : 1.0,
    'RDNOISE' : image_header['READNSEC'], 
    'GAIN' : image_header['CCDGAIN']})
fits.writeto('./bg_subtracted_abell2744_test_image.fits', image.data, header=image_header, overwrite=True)

rms_header = fits.getheader(url.replace('drz', 'rms'))
rms_header.update(image.wcs.to_header())
fits.writeto('./bg_subtracted_abell2744_test_rms.fits', rms.data, header=rms_header, overwrite=True)

## Make Source Catalog 

We will use the sigma clipped std as a threshold at the segmentation and deblending steps.


In [ ]:
from astropy.stats import sigma_clipped_stats
image_mean, image_median, image_stddev = sigma_clipped_stats(image.data, sigma=3)

Here we identity sources in the input image.

In [ ]:
from petrofit.segmentation import make_catalog, plot_segments

threshold = image_stddev * 2

# Define smoothing kernel
kernel_size = 3
fwhm = 3

# Min Source size (area)
npixels = 2**2


cat, segm, segm_deblend = make_catalog(    
    image.data, 
    threshold=threshold, 
    deblend=True,                 
    kernel_size=kernel_size,                  
    fwhm=fwhm, 
    npixels=npixels,
    contrast=0.00,
    plot=True, vmax=vmax, vmin=vmin
)

plt.show()

# Display source properties
print("Num of Targets:", len(cat))

In [ ]:
from petrofit.photometry import order_cat
from petrofit.utils import plot_target
# Sort and select object of interest in the catalog
sorted_idx_list = order_cat(cat, key='area', reverse=True)
idx = sorted_idx_list[2] # index 0 is largest 
source = cat[idx]  # get source from the catalog 

plot_target(
    position=(source.maxval_xindex, source.maxval_yindex), 
    image=image.data, 
    size=max(image.data.shape)//2, 
    vmax=vmax, 
    vmin=vmin
)

## Save Mask

In [ ]:
from petrofit.segmentation import masked_segm_image
pixel_mask = masked_segm_image(source, np.zeros_like(image.data), segm_deblend)
pixel_mask[np.where(np.isnan(pixel_mask))] = 1
plt.imshow(pixel_mask)

fits.writeto('./bg_subtracted_abell2744_test_mask.fits', pixel_mask, overwrite=True)

In [ ]:
segm_deblend

In [ ]:
import pickle

In [ ]:
pickle.dump(segm_deblend, open( "bg_subtracted_abell2744_test_segm_deblend.p", "wb" ))